In [3]:
import pandas as pd
import numpy as np
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [4]:
data = pd.read_csv('train.csv')
data.pop('id');
data_train, data_test = train_test_split(data, test_size = 0.1)

In [5]:
def getdata(data):
    X = []
    Y = []
    grouped = data.groupby('target')
    for key, item in grouped:
        tmp = grouped.get_group(key)
        X.append(tmp[tmp.columns[0:93]].values)    
        Y.append(tmp[tmp.columns[93]].values)
    return X,Y

In [6]:
def concatenate(X,Y):
    x = np.concatenate((X),axis = 0)
    y = np.concatenate((Y),axis = 0)
    return x,y 

In [7]:
def decision_tree(X,Y):
    from sklearn import tree
    dt = tree.DecisionTreeClassifier(min_samples_split= 20, random_state=99)
    dt.fit(X,Y)
    score = cross_val_score(dt, X, Y, cv = 5)
    return score,dt

In [8]:
def svm(X,Y):
    from sklearn import svm
    svm = svm.SVC()
    svm.fit(X_train, y_train)
    score = cross_val_score(svm, X, Y, cv = 5)
    return score,svm

In [9]:
def xgboost(X,Y):
    import xgboost as xgb
    gbm = xgb.XGBClassifier(max_depth=20, n_estimators=300, learning_rate=0.05).fit(X_train,y_train)
    score = cross_val_score(gbm, X, Y, cv = 5)
    return score,gbm

In [10]:
def logi_reg(X,Y):
    from sklearn import linear_model
    log = linear_model.LogisticRegression(C=1e5).fit(X_train,y_train)
    score = cross_val_score(log, X, Y, cv = 5)
    return score,log

In [11]:
def random_forest(X,Y):
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators = 100)
    forest = forest.fit(X_train,y_train)
    score = cross_val_score(forest, X, Y, cv = 5)
    return score,forest

In [12]:
def bayes_naive_gaussien(X,Y):
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb = gnb.fit(X,Y)
    score = cross_val_score(gnb, X,Y, cv = 5)
    return score

In [13]:
def rate_correction (Y_pred,Y_true):
    rate = 0
    l = len(Y_pred)
    for i in range(l):
        if Y_pred[i] == Y_true[i]:
            rate = rate + 1
    return rate/l

In [12]:
def normalisation(X):
    for i in range(len(X)):
        X[i] = (X[i]-np.mean(X[i],axis=0))/np.std(X[i], axis=0)
    return X

In [14]:
#=========Read the data set=====================================
X,Y=getdata(data)
X,Y=concatenate(X,Y)
X_train,y_train = getdata(data_train)
X_train,y_train = concatenate(X_train,y_train)
X_test,y_test = getdata(data_test)
X_test,y_test = concatenate(X_test,y_test)
print 'The shape of the train set:'+ str(X.shape)
print 'The shape of the train set:'+ str(X_train.shape)
print 'The shape of the test set:'+ str(X_test.shape)

The shape of the train set:(61878, 93)
The shape of the train set:(55690, 93)
The shape of the test set:(6188, 93)


In [16]:
#Result of the decision tree model

score_dt  = decision_tree(X,Y)
print score_dt

(array([ 0.72127969,  0.71867173,  0.71800259,  0.72365869,  0.72152615]), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            presort=False, random_state=99, splitter='best'))


In [47]:
#Result of the logistic regression model
score_log  = logi_reg(X,Y)
score_log

(array([ 0.727639  ,  0.73769169,  0.7283751 ,  0.74029126,  0.72712551]),
 LogisticRegression(C=100000.0, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False))

In [49]:
score_xg = xgboost(X,Y)
score_xg

ImportError: No module named xgboost

In [17]:
#Result of the random forest model
score_rf  = random_forest(X,Y)
score_rf

(array([ 0.8061884 ,  0.80649592,  0.80769231,  0.81286361,  0.8126263 ]),
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
             verbose=0, warm_start=False))

In [51]:
#Result of the bayen naive model
score_bn = bayes_naive_gaussien(X,Y)
score_bn

array([ 0.52699436,  0.51977401,  0.59660469,  0.57281553,  0.60566802])

In [39]:
#Result of the svm
score_svm  = svm(X,Y)

KeyboardInterrupt: 

In [44]:
score_svm

(array([ 0.7851026 ,  0.78225741,  0.78595669,  0.79064318,  0.78465767]),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False))

# Tuning hyper-parameters as an estimator
This part is to improve the hyperparameters in using grid search. Grid search is implemented and the random search as well to compare with the runtime. The random search has the advantage of runtime and more paramter ranges in the cost of comparetively lower score.

In [15]:
from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# build a classifier
clf = RandomForestClassifier(n_estimators=20)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}


# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, Y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

## use a full grid over all parameters
#param_grid = {"max_depth": [3, None],
#              "max_features": [1, 3, 10],
#              "min_samples_split": [1, 3, 10],
#              "min_samples_leaf": [1, 3, 10],
#              "bootstrap": [True, False],
#              "criterion": ["gini", "entropy"]}

## run grid search
#grid_search = GridSearchCV(clf, param_grid=param_grid)
#start = time()
#grid_search.fit(X, Y)

#print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
#      % (time() - start, len(grid_search.cv_results_['params'])))
#report(grid_search.cv_results_)



RandomizedSearchCV took 117.82 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.798 (std: 0.002)
Parameters: {'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 6, 'criterion': 'gini', 'max_features': 9, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.793 (std: 0.004)
Parameters: {'bootstrap': True, 'min_samples_leaf': 1, 'min_samples_split': 4, 'criterion': 'gini', 'max_features': 9, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.791 (std: 0.002)
Parameters: {'bootstrap': False, 'min_samples_leaf': 5, 'min_samples_split': 8, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}



# An outdata of comparison between random search and grid search.
RandomizedSearchCV took 95.47 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.792 (std: 0.001)
Parameters: {'bootstrap': False, 'min_samples_leaf': 3, 'min_samples_split': 1, 'criterion': 'gini', 'max_features': 8, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.776 (std: 0.001)
Parameters: {'bootstrap': False, 'min_samples_leaf': 10, 'min_samples_split': 4, 'criterion': 'gini', 'max_features': 9, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.775 (std: 0.001)
Parameters: {'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': 4, 'max_depth': None}

GridSearchCV took 892.67 seconds for 216 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.798 (std: 0.001)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 10, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.798 (std: 0.002)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 3, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.797 (std: 0.001)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 1, 'criterion': 'gini', 'max_features': 10, 'max_depth': None}

# Make the submission file to test the score online

In [16]:
data_test = pd.read_csv('test.csv')
data_test.pop('id');
X_submission=data_test.values

In [17]:
X_submission=data_test.values

In [84]:
def submission(estimator,test):
    y_pred=estimator.predict(test)
    print 'The prediction is like: ' + str(y_pred[1:10])
    y_pred=[int(Class[-1:]) for Class in y_pred]
    print 'The prediction aftre transform is like: ' + str(y_pred[1:10])
    l_y=len(y_pred)
    print 'The size of the test data is: '+str(l_y)
    y_pred=np.asarray(y_pred)
    Value=np.zeros([l_y,10])
    Value[:,0]=range(1,l_y+1)
    for i in range(1,10):
        Value[y_pred==i,i]=1 
    Submission=pd.DataFrame(Value,columns=['id','Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])
    Submission=Submission.astype(int)
    Submission.to_csv('Submission.csv',index=False)
    return Submission

In [85]:
Submission=submission(random_search.best_estimator_,X_submission)

The prediction is like: ['Class_6' 'Class_6' 'Class_2' 'Class_9' 'Class_3' 'Class_8' 'Class_2'
 'Class_3' 'Class_2']
The prediction aftre transform is like: [6, 6, 2, 9, 3, 8, 2, 3, 2]
The size of the test data is: 144368


In [86]:
Submission

,id,Classe_1,Classe_2,Classe_3,Classe_4,Classe_5,Classe_6,Classe_7,Classe_8,Classe_9
0,1,0,0,0,1,0,0,0,0,0
1,2,0,0,0,0,0,1,0,0,0
2,3,0,0,0,0,0,1,0,0,0
3,4,0,1,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,1
5,6,0,0,1,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,1,0
7,8,0,1,0,0,0,0,0,0,0
8,9,0,0,1,0,0,0,0,0,0
9,10,0,1,0,0,0,0,0,0,0
